# Speech Commands Dataset - Deep Dive with Visualizations

## Introduction

In this notebook, we'll work with **real audio data** from the Speech Commands Dataset. Unlike our synthetic phoneme data from Notebook 1, this dataset contains actual human speech with natural co-articulation effects, making it perfect for demonstrating RNN superiority.

### Learning Objectives

1. Download and explore the Speech Commands Digits dataset
2. Understand the complete audio processing pipeline
3. Extract MFCC features from real speech
4. Visualize temporal patterns in spoken digits
5. Prepare data for RNN training

### Dataset Overview

**Speech Commands Dataset (Digits Subset)**
- **Words**: "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"
- **Size**: ~200MB (10 digits)
- **Samples**: ~20,000 audio clips (~2,000 per digit)
- **Duration**: 1 second per clip
- **Sample Rate**: 16kHz
- **Speakers**: Multiple speakers with various accents

### Why This Dataset?

Real speech has properties that make temporal modeling essential:
- **Co-articulation**: Sounds blend together naturally
- **Speaking rate variation**: Different speeds
- **Accent diversity**: Different pronunciations
- **Prosody**: Natural rhythm and intonation

These properties create **ambiguous frames** that require temporal context to classify correctly - perfect for demonstrating RNN advantages!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
import os
import urllib.request
import tarfile
from pathlib import Path
from tqdm import tqdm
import IPython.display as ipd
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Librosa version: {librosa.__version__}")

## 1. Audio Processing Pipeline Overview

Before downloading the data, let's understand the complete pipeline for converting raw audio to features suitable for RNN training.

### Complete Pipeline Diagram

```
┌─────────────────────────────────────────────────────────────────────┐
│                    AUDIO PROCESSING PIPELINE                        │
└─────────────────────────────────────────────────────────────────────┘

Step 1: RAW AUDIO
┌─────────────────┐
│  "seven.wav"    │  • Duration: 1 second
│  16,000 samples │  • Sample rate: 16kHz
│  Mono channel   │  • Format: WAV
└────────┬────────┘
         │
         ▼
Step 2: FRAMING
┌─────────────────┐
│   Split into    │  • Window: 25ms (400 samples)
│   overlapping   │  • Hop: 10ms (160 samples)
│   frames        │  • Result: ~100 frames
└────────┬────────┘
         │
         ▼
Step 3: WINDOWING
┌─────────────────┐
│  Apply Hamming  │  • Reduces spectral leakage
│  window to each │  • Smooth frame boundaries
│  frame          │  • w[n] = 0.54 - 0.46·cos(2πn/N)
└────────┬────────┘
         │
         ▼
Step 4: FFT (Fast Fourier Transform)
┌─────────────────┐
│  Time → Freq    │  • 512-point FFT
│  domain         │  • Power spectrum
│  conversion     │  • |FFT|² → Energy per frequency
└────────┬────────┘
         │
         ▼
Step 5: MEL FILTERBANK
┌─────────────────┐
│  Apply 40 mel   │  • Mimics human hearing
│  filters        │  • More resolution at low freq
│                 │  • Less resolution at high freq
└────────┬────────┘
         │
         ▼
Step 6: LOG COMPRESSION
┌─────────────────┐
│  log(mel_spec)  │  • Mimics loudness perception
│                 │  • Compresses dynamic range
│                 │  • log₁₀(x + ε)
└────────┬────────┘
         │
         ▼
Step 7: DCT (Discrete Cosine Transform)
┌─────────────────┐
│  Decorrelate    │  • Keep first 13 coefficients
│  features       │  • MFCCs = DCT(log mel spectrum)
│                 │  • Compact representation
└────────┬────────┘
         │
         ▼
FINAL OUTPUT
┌─────────────────┐
│  MFCC Matrix    │  • Shape: (100 frames, 13 coeffs)
│  100 × 13       │  • Ready for RNN input
│                 │  • Temporal sequence preserved
└─────────────────┘
```

### Key Parameters

| Parameter | Value | Reason |
|-----------|-------|--------|
| Sample Rate | 16kHz | Standard for speech (captures up to 8kHz) |
| Frame Length | 25ms | Captures phonetic information |
| Frame Hop | 10ms | 60% overlap for smooth transitions |
| FFT Size | 512 | Power of 2, sufficient resolution |
| Mel Filters | 40 | Standard for speech recognition |
| MFCC Coeffs | 13 | Captures essential spectral envelope |

### Why MFCCs?

MFCCs (Mel-Frequency Cepstral Coefficients) are ideal for speech because:
1. **Perceptually motivated**: Mel scale matches human hearing
2. **Compact**: 13 numbers capture essential information
3. **Decorrelated**: DCT removes redundancy
4. **Robust**: Less sensitive to noise than raw spectrum
5. **Standard**: Widely used in speech recognition

## 2. Download Speech Commands Dataset

We'll download only the digit folders to keep the size manageable (~200MB instead of 2GB).

In [ ]:
# Configuration
DATA_DIR = Path('speech_commands_data')
DATASET_URL = 'http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz'
DIGITS = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']

def download_speech_commands():
    """Download and extract Speech Commands dataset."""
    
    # Create data directory
    DATA_DIR.mkdir(exist_ok=True)
    
    # Check if already downloaded
    if all((DATA_DIR / digit).exists() for digit in DIGITS):
        print("✓ Dataset already downloaded!")
        return
    
    print("Downloading Speech Commands dataset...")
    print("This may take a few minutes (~2GB download)")
    
    # Download
    tar_path = DATA_DIR / 'speech_commands.tar.gz'
    
    if not tar_path.exists():
        urllib.request.urlretrieve(DATASET_URL, tar_path)
        print("✓ Download complete!")
    
    # Extract only digit folders
    print("\nExtracting digit folders...")
    with tarfile.open(tar_path, 'r:gz') as tar:
        members = [m for m in tar.getmembers() 
                  if any(m.name.startswith(digit + '/') for digit in DIGITS)]
        
        for member in tqdm(members, desc="Extracting"):
            tar.extract(member, DATA_DIR)
    
    print("\n✓ Extraction complete!")
    
    # Clean up
    if tar_path.exists():
        tar_path.unlink()
        print("✓ Cleaned up temporary files")

# Download dataset
download_speech_commands()

# Count files
print("\n" + "="*50)
print("DATASET STATISTICS")
print("="*50)
for digit in DIGITS:
    digit_path = DATA_DIR / digit
    if digit_path.exists():
        n_files = len(list(digit_path.glob('*.wav')))
        print(f"{digit:>6}: {n_files:>5} samples")

total_files = sum(len(list((DATA_DIR / digit).glob('*.wav'))) 
                 for digit in DIGITS if (DATA_DIR / digit).exists())
print("="*50)
print(f"{'TOTAL':>6}: {total_files:>5} samples")
print("="*50)

## 3. Explore Raw Audio

Let's load and visualize some audio samples to understand what we're working with.

In [ ]:
def load_random_sample(digit, sr=16000):
    """Load a random audio sample for a given digit."""
    digit_path = DATA_DIR / digit
    audio_files = list(digit_path.glob('*.wav'))
    
    if not audio_files:
        raise ValueError(f"No audio files found for digit: {digit}")
    
    # Pick random file
    audio_file = np.random.choice(audio_files)
    
    # Load audio
    audio, sr = librosa.load(audio_file, sr=sr)
    
    return audio, sr, audio_file.name

# Load samples for visualization
sample_digits = ['zero', 'five', 'seven']
samples = {}

for digit in sample_digits:
    audio, sr, filename = load_random_sample(digit)
    samples[digit] = {'audio': audio, 'sr': sr, 'filename': filename}
    print(f"Loaded {digit}: {filename}")

print(f"\n✓ Loaded {len(samples)} samples")
print(f"Sample rate: {sr} Hz")
print(f"Duration: {len(audio)/sr:.2f} seconds")
print(f"Samples per audio: {len(audio)}")

### 3.1 Listen to Audio Samples

Let's listen to the actual audio to understand what we're analyzing:

In [ ]:
# Play audio samples
for digit, data in samples.items():
    print(f"\n🔊 Playing: {digit.upper()}")
    print(f"   File: {data['filename']}")
    display(ipd.Audio(data['audio'], rate=data['sr']))

### 3.2 Visualize Waveforms

Waveforms show amplitude over time - the raw representation of sound.

In [ ]:
fig, axes = plt.subplots(len(sample_digits), 1, figsize=(14, 10))

for idx, (digit, data) in enumerate(samples.items()):
    audio = data['audio']
    sr = data['sr']
    
    # Time axis
    time = np.linspace(0, len(audio)/sr, len(audio))
    
    # Plot waveform
    axes[idx].plot(time, audio, linewidth=0.5, alpha=0.8)
    axes[idx].set_ylabel('Amplitude')
    axes[idx].set_title(f'Waveform: "{digit.upper()}" - {data["filename"]}', 
                       fontsize=12, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xlim([0, len(audio)/sr])
    
    # Add zero line
    axes[idx].axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)

axes[-1].set_xlabel('Time (seconds)')
plt.tight_layout()
plt.show()

print("\n📊 Waveform Observations:")
print("  • Different digits have different temporal patterns")
print("  • Some digits are longer/shorter than others")
print("  • Amplitude varies throughout the utterance")
print("  • Silence at beginning/end of some samples")

## 4. Temporal Pattern Analysis

Let's analyze how speech unfolds over time using a detailed example.

### 4.1 Phonetic Breakdown of "SEVEN"

```
┌────────────────────────────────────────────────────────────────────┐
│              TEMPORAL BREAKDOWN: "SEVEN"                           │
└────────────────────────────────────────────────────────────────────┘

Time:     0ms      200ms     400ms     600ms     800ms    1000ms
          │         │         │         │         │         │
          ▼         ▼         ▼         ▼         ▼         ▼
Sound:   [s]   →   [e]   →   [v]   →   [ə]   →   [n]   → [silence]
         │         │         │         │         │         │
Phoneme: /s/       /ɛ/       /v/       /ə/       /n/      
         │         │         │         │         │         │
Type:    Fric.     Vowel     Fric.     Vowel     Nasal    Silent
         │         │         │         │         │         │
Energy:  ▂▂▂▂▂    ▅▅▅▅▅    ▃▃▃▃▃    ▅▅▅▅▅    ▃▃▃▃▃    ▁▁▁▁▁
         │         │         │         │         │         │
Freq:    High      Mid       Low       Mid       Low      Silent
         4-8kHz    1-3kHz    0.5-2kHz  1-3kHz    0.5-1kHz  
         │         │         │         │         │         │
MFCC:   Frame1    Frame2    Frame3    Frame4    Frame5   Frame6...
        [13]      [13]      [13]      [13]      [13]     [13]
         │         │         │         │         │         │
         └─────────┴─────────┴─────────┴─────────┴─────────┘
                            Sequence for RNN
```

### Key Observations:

1. **Temporal Structure**: Speech has clear temporal organization
2. **Phoneme Transitions**: Sounds blend together (co-articulation)
3. **Energy Variation**: Different phonemes have different energy levels
4. **Frequency Content**: Each phoneme has characteristic frequencies
5. **Sequential Nature**: Order matters - "seven" ≠ "neves"

### Why RNNs Are Necessary:

**Problem with Frame-by-Frame (DNN):**
- Frame 2 (vowel /ɛ/) could be from "seven", "ten", "eleven", "set", etc.
- Without context, impossible to disambiguate

**Solution with Sequential (RNN):**
- Frame 1 (/s/) + Frame 2 (/ɛ/) → Likely "seven" or "set"
- Frame 3 (/v/) confirms → Must be "seven"!
- Context resolves ambiguity

### 4.2 Visualize Spectrograms

Spectrograms show frequency content over time - revealing the temporal structure of speech.

In [ ]:
fig, axes = plt.subplots(len(sample_digits), 1, figsize=(14, 12))

for idx, (digit, data) in enumerate(samples.items()):
    audio = data['audio']
    sr = data['sr']
    
    # Compute spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
    
    # Plot
    img = librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz', 
                                    ax=axes[idx], cmap='viridis')
    axes[idx].set_title(f'Spectrogram: "{digit.upper()}"', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Frequency (Hz)')
    axes[idx].set_ylim([0, 8000])
    
    # Add colorbar
    plt.colorbar(img, ax=axes[idx], format='%+2.0f dB')

axes[-1].set_xlabel('Time (seconds)')
plt.tight_layout()
plt.show()

print("\n📊 Spectrogram Observations:")
print("  • Horizontal bands = Formants (resonances of vocal tract)")
print("  • Bright regions = High energy at those frequencies")
print("  • Dark regions = Low energy")
print("  • Temporal patterns visible - different for each digit")
print("  • Transitions between phonemes create characteristic patterns")

## 5. MFCC Feature Extraction

Now let's extract MFCC features - the input for our RNN.

### 5.1 MFCC Computation Steps

```
┌──────────────────────────────────────────────────────────────┐
│                  MFCC EXTRACTION PROCESS                     │
└──────────────────────────────────────────────────────────────┘

Input: audio[16000] (1 second at 16kHz)
   │
   ▼
┌─────────────────┐
│ 1. Frame Audio  │  window=25ms, hop=10ms
│    into windows │  → 100 frames
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ 2. Apply        │  Hamming window
│    Windowing    │  → Smooth boundaries
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ 3. Compute FFT  │  512-point FFT
│                 │  → Power spectrum
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ 4. Mel Filter   │  40 triangular filters
│    Bank         │  → Mel spectrum
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ 5. Log          │  log₁₀(mel_spec + ε)
│    Compression  │  → Log mel spectrum
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│ 6. DCT          │  Keep first 13 coeffs
│                 │  → 13 MFCCs
└────────┬────────┘
         │
         ▼
Output: mfcc[100, 13] (100 frames × 13 coefficients)
```

In [ ]:
def extract_mfcc(audio, sr=16000, n_mfcc=13, n_fft=512, hop_length=160):
    """Extract MFCC features from audio.
    
    Args:
        audio: Audio signal
        sr: Sample rate
        n_mfcc: Number of MFCC coefficients
        n_fft: FFT window size
        hop_length: Hop length between frames
    
    Returns:
        mfcc: MFCC features (n_frames, n_mfcc)
    """
    # Extract MFCCs
    mfcc = librosa.feature.mfcc(
        y=audio,
        sr=sr,
        n_mfcc=n_mfcc,
        n_fft=n_fft,
        hop_length=hop_length
    )
    
    # Transpose to (time, features)
    mfcc = mfcc.T
    
    return mfcc

# Extract MFCCs for our samples
mfccs = {}
for digit, data in samples.items():
    mfcc = extract_mfcc(data['audio'], data['sr'])
    mfccs[digit] = mfcc
    print(f"{digit:>6}: MFCC shape = {mfcc.shape} (frames × coefficients)")

print("\n✓ MFCC extraction complete!")

### 5.2 Visualize MFCC Features

MFCCs show the spectral envelope over time - the key features for speech recognition.

In [ ]:
fig, axes = plt.subplots(len(sample_digits), 1, figsize=(14, 12))

for idx, (digit, mfcc) in enumerate(mfccs.items()):
    # Plot MFCC heatmap
    im = axes[idx].imshow(mfcc.T, aspect='auto', origin='lower', 
                          cmap='RdBu_r', interpolation='nearest')
    axes[idx].set_ylabel('MFCC Coefficient')
    axes[idx].set_title(f'MFCC Features: "{digit.upper()}"', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_yticks(range(13))
    axes[idx].set_yticklabels([f'C{i}' for i in range(13)])
    
    # Add colorbar
    plt.colorbar(im, ax=axes[idx], label='MFCC Value')

axes[-1].set_xlabel('Frame Number (time →)')
plt.tight_layout()
plt.show()

print("\n📊 MFCC Observations:")
print("  • Each row = one MFCC coefficient over time")
print("  • Each column = one time frame (10ms)")
print("  • C0 (bottom) = Energy/loudness")
print("  • C1-C12 = Spectral shape information")
print("  • Temporal patterns visible - different for each digit")
print("  • This is the input sequence for our RNN!")

## 6. Prepare Dataset for Training

Now let's process all audio files and create train/validation/test splits.

In [ ]:
def load_dataset(data_dir, digits, max_samples_per_digit=None, sr=16000):
    """Load and process entire dataset.
    
    Args:
        data_dir: Path to data directory
        digits: List of digit names
        max_samples_per_digit: Maximum samples per digit (None = all)
        sr: Sample rate
    
    Returns:
        X: MFCC features (n_samples, n_frames, n_mfcc)
        y: Labels (n_samples,)
        label_map: Dictionary mapping digit names to indices
    """
    X_list = []
    y_list = []
    label_map = {digit: idx for idx, digit in enumerate(digits)}
    
    print("Loading dataset...")
    for digit in digits:
        digit_path = data_dir / digit
        audio_files = list(digit_path.glob('*.wav'))
        
        # Limit samples if specified
        if max_samples_per_digit:
            audio_files = audio_files[:max_samples_per_digit]
        
        print(f"  Processing {digit}: {len(audio_files)} samples")
        
        for audio_file in tqdm(audio_files, desc=f"  {digit}", leave=False):
            try:
                # Load audio
                audio, _ = librosa.load(audio_file, sr=sr)
                
                # Extract MFCCs
                mfcc = extract_mfcc(audio, sr)
                
                # Store
                X_list.append(mfcc)
                y_list.append(label_map[digit])
                
            except Exception as e:
                print(f"    Error loading {audio_file.name}: {e}")
                continue
    
    # Convert to arrays
    X = np.array(X_list)
    y = np.array(y_list)
    
    return X, y, label_map

# Load dataset (limit to 200 samples per digit for faster processing)
print("Loading Speech Commands dataset...")
print("(Using 200 samples per digit for faster training)\n")

X, y, label_map = load_dataset(DATA_DIR, DIGITS, max_samples_per_digit=200)

print(f"\n✓ Dataset loaded!")
print(f"  X shape: {X.shape} (samples, frames, mfcc_coeffs)")
print(f"  y shape: {y.shape} (samples,)")
print(f"  Label map: {label_map}")

In [ ]:
# Create train/validation/test splits
from sklearn.model_selection import train_test_split

# First split: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: 75% train, 25% val (of the 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print("Dataset splits:")
print(f"  Train: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"  Val:   {X_val.shape[0]} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"  Test:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Check class distribution
print("\nClass distribution (train):")
for digit, idx in label_map.items():
    count = np.sum(y_train == idx)
    print(f"  {digit:>6}: {count:>3} samples")

## 7. Save Processed Data

Save the processed data for use in the next notebook.

In [ ]:
# Save processed data
np.savez_compressed(
    'speech_commands_processed.npz',
    X_train=X_train,
    X_val=X_val,
    X_test=X_test,
    y_train=y_train,
    y_val=y_val,
    y_test=y_test,
    label_map=label_map,
    digits=DIGITS
)

print("✓ Processed data saved to 'speech_commands_processed.npz'")
print("\nReady for RNN training in Notebook 2B!")

## Summary

### What We Accomplished:

✅ **Downloaded real speech data** - Speech Commands digits (~200MB)  
✅ **Understood the pipeline** - From raw audio to MFCC features  
✅ **Visualized temporal patterns** - Waveforms, spectrograms, MFCCs  
✅ **Extracted features** - 13 MFCCs per frame, ~100 frames per audio  
✅ **Prepared dataset** - Train/val/test splits ready for training

### Key Insights:

1. **Real speech has temporal structure** - Phonemes unfold over time
2. **Co-articulation creates ambiguity** - Individual frames are ambiguous
3. **Context is essential** - Need previous frames to disambiguate
4. **MFCCs capture essential information** - Compact, perceptually motivated
5. **Sequences matter** - Order of frames is crucial

### Next Steps:

In **Notebook 2B**, we'll:
1. Train a DNN baseline (frame-by-frame classification)
2. Train an RNN (sequential classification)
3. Compare performance (expect 15-25% improvement)
4. Analyze where each model succeeds/fails
5. Visualize what the RNN learns

**The real data will clearly demonstrate why RNNs are necessary for sequence tasks!** 🎯